In [1]:
%display latex

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os

In [3]:
#Define space-time manifold and coordinates

n = 3
M = Manifold(1+n, 'M', structure='Lorentzian')
X.<t,r,th,ph> = M.chart(r"t r:(0,+oo) th:(0,pi):\theta ph:(0,2*pi):\phi")

In [4]:
eps = var('eps', latex_name='\\varepsilon')

h = function('h')(r,th)
k = function('k')(r,th)
m = function('m')(r,th)
omega = function('omega', latex_name="\\omega")(r,th)

nu = function('nu', latex_name='\\nu')(r)
lamb = function('lamb', latex_name='\\lambda')(r)

Omegae = function('Omegaevalue', latex_name = "\\Omega_e")(eps)
Ne = function('Ne', latex_name = "N_e")(r,eps)

In [5]:
gdd = M.tensor_field(0,2, name='gdd')
gdd[0,0] = -exp(nu)
gdd[1,1] = exp(lamb)
gdd[2,2] = r^2
gdd[3,3] = r^2*sin(th)^2

K1 = M.tensor_field(0,2, name='K1')
K1[0,3] = omega*r^2*sin(th)^2
K1[3,0] = omega*r^2*sin(th)^2

K2 = M.tensor_field(0,2, name='K2')
K2[0,0] = -4*exp(nu)*h + 2*r^2*sin(th)^2*omega^2
K2[1,1] = 4*exp(lamb)*m
K2[2,2] = 4*k*r^2
K2[3,3] = 4*k*r^2*sin(th)^2

gedd = M.metric()
for i in range(0,n+1):
    for j in range(0,n+1):
        gedd[i,j] = gdd[i,j] + eps*K1[i,j] + 1/2*eps^2*K2[i,j]

geuu = gedd.inverse()
ricci = gedd.ricci()
s_curv = geuu['^{ab}']*ricci['_ab']
eins = ricci - gdd*s_curv/2                                                                             #G_\mu\nu e

deins = M.tensor_field(0,2,'deins')
ddeins = M.tensor_field(0,2,'ddeins')
for i in range(0,n+1):
    for j in range(0,n+1):
        deins[i,j] = diff(eins[i,j].expr(),eps)
        ddeins[i,j] = diff(eins[i,j].expr(),eps,eps)

In [6]:
show(LatexExpr("g_{\\varepsilon}\\rvert_{\\varepsilon=0} = "), gdd[:])
show(LatexExpr("K_1 = "), K1[:])
show(LatexExpr("K_2 = "), K2[:])
show(LatexExpr("g_{\\varepsilon} = "), gedd[:])

g_{\varepsilon}\rvert_{\varepsilon=0} =  [     -e^nu(r)             0             0             0]
[            0     e^lamb(r)             0             0]
[            0             0           r^2             0]
[            0             0             0 r^2*sin(th)^2]

K_1 =  [                         0                          0                          0 r^2*omega(r, th)*sin(th)^2]
[                         0                          0                          0                          0]
[                         0                          0                          0                          0]
[r^2*omega(r, th)*sin(th)^2                          0                          0                          0]

K_2 =  [2*r^2*omega(r, th)^2*sin(th)^2 - 4*e^nu(r)*h(r, th)                                                   0                                                   0                                                   0]
[                                                  0                                4*e^lamb(r)*m(r, th)                                                   0                                                   0]
[                                                  0                                                   0                                      4*r^2*k(r, th)                                                   0]
[                                                  0                                                   0                                                   0                            4*r^2*k(r, th)*sin(th)^2]

g_{\varepsilon} =  [eps^2*r^2*omega(r, th)^2*sin(th)^2 - 2*eps^2*e^nu(r)*h(r, th) - e^nu(r)                                                                       0                                                                       0                                          eps*r^2*omega(r, th)*sin(th)^2]
[                                                                      0                                  2*eps^2*e^lamb(r)*m(r, th) + e^lamb(r)                                                                       0                                                                       0]
[                                                                      0                                                                       0                                              2*eps^2*r^2*k(r, th) + r^2                                                                       0]
[                                         eps*r^2*omega(r, th)*sin(th)^2                                                                       0                                                                       0                                  (2*eps^2*r^2*k(r, th) + r^2)*sin(th)^2]

In [7]:
#4-velocities of the fluid for any epsilon

ue = M.vector_field(name="ue")
ue[0] = Ne; ue[1] = 0; ue[2] = 0; ue[3] = Ne*Omegae

u = M.vector_field(name="u")
u[0] = exp(-nu/2); u[1] =  0; u[2] =  0; u[3] =  0

Nevalue = solve(u[0].expr()==ue[0].expr(), Ne)[0].rhs()                                                    #Eq 4.20
Omegaevalue = solve(u[3].expr()==ue[3].expr(), Omegae)[0].rhs()                                            #Eq 4.20

In [8]:
show(LatexExpr("N_{\\varepsilon}\\rvert_{\\varepsilon=0} = "), Nevalue)
show(LatexExpr("\\Omega_{\\varepsilon}\\rvert_{\\varepsilon=0} = "), Omegaevalue)

N_{\varepsilon}\rvert_{\varepsilon=0} =  e^(-1/2*nu(r))

\Omega_{\varepsilon}\rvert_{\varepsilon=0} =  0

In [9]:
u1 = M.vector_field(name="u1",latex_name="u^{(1)}")
u2 = M.vector_field(name="u2",latex_name="u^{(2)}")

u10 = function('u10',latex_name="u^{(1)0}")(r)
u20 = function('u20',latex_name="u^{(2)0}")(r)

Omega1 = function('Omega1', latex_name="\\Omega^{(1)}")(eps)
Omega2 = function('Omega2', latex_name="\\Omega^{(2)}")(eps)

s0 = eps == 0
s1 = Ne.subs(eps==0) == Nevalue
s2 = Omegae.subs(eps==0) == Omegaevalue
s3 = diff(Ne,eps).subs(eps==0) == u10
s4 = diff(Omegae,eps).subs(eps==0) == Omega1
s5 = diff(Ne,eps,eps).subs(eps==0) == u20
s6 = diff(Omegae,eps,eps).subs(eps==0) == Omega2

sustituciones = (s0,s1,s2,s3,s4,s5,s6)

for i in range(0,n+1):
    u1[i] = diff(ue[i].expr(),eps).subs(sustituciones)
    u2[i] = diff(ue[i].expr(),eps,eps).subs(sustituciones)

Ue = ue.down(gedd)
geUeue = Ue['_a']*ue['^a']
dgeUeue = diff(geUeue.expr(), eps).subs(sustituciones) == 0
u10value = solve(dgeUeue, u10)[0].rhs()
s7 = u10 == u10value
u1.apply_map(lambda cmp: cmp.subs(u10==u10value))                                                          #Eq 4.21

ddgeUeue = diff(geUeue.expr(), eps, eps).subs(sustituciones) == 0
u20value = solve(ddgeUeue, u20)[0].rhs()
s8 = u20 == u20value
u2.apply_map(lambda cmp: cmp.subs(u20==u20value).subs(u10==u10value))                                      #Eq 4.22

In [10]:
show(u1.display())
show(u2.display())

u1 = Omega1(eps)*e^(-1/2*nu(r)) ∂/∂ph

u2 = ((r^2*Omega1(eps)^2 + 2*r^2*Omega1(eps)*omega(r, th) + r^2*omega(r, th)^2)*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-3/2*nu(r)) ∂/∂t + Omega2(eps)*e^(-1/2*nu(r)) ∂/∂ph

In [11]:
deins.apply_map(lambda cmp: cmp.subs(sustituciones))                                             #d/de(G_\mu\nu)|e=0
ddeins.apply_map(lambda cmp: cmp.subs(sustituciones))                                        #d^2/de^2(G_\mu\nu)|e=0

In [12]:
Ee = function('Ee', latex_name="E_{\\varepsilon}")(r,eps)
Pe = function('Pe', latex_name="P_{\\varepsilon}")(r,eps)

eins0 = M.tensor_field(0,2, 'eins0', latex_name='G_0')
Tedd = M.tensor_field(0,2,name='Tedd',latex_name="Tdd_{\\varepsilon}")
Tdd = M.tensor_field(0,2, 'Tdd')
for i in range(0,n+1):
    for j in range(0,n+1):
        eins0[i,j] = eins[i,j]
        Tedd[i,j] = (Ee + Pe)*Ue[i]*Ue[j] + Pe*gedd[i,j]                                                 #T_\mu\nu e
        Tdd[i,j] = Tedd[i,j]

eins0.apply_map(lambda cmp: cmp.subs(sustituciones))                                                       #G_\mu\nu
Tdd.apply_map(lambda cmp: cmp.subs(sustituciones))                                                         #T_\mu\nu

In [13]:
show(LatexExpr("T_{\\varepsilon}\\rvert_{\\varepsilon=0} = "), Tdd[:])

T_{\varepsilon}\rvert_{\varepsilon=0} =  [      Ee(r, 0)*e^nu(r)                      0                      0                      0]
[                     0     Pe(r, 0)*e^lamb(r)                      0                      0]
[                     0                      0           r^2*Pe(r, 0)                      0]
[                     0                      0                      0 r^2*Pe(r, 0)*sin(th)^2]

In [14]:
eins0[0,0].expr()

(r*e^(3*nu(r))*diff(lamb(r), r) + (e^lamb(r) - 1)*e^(3*nu(r)))*e^(-lamb(r) - 2*nu(r))/r^2

In [41]:
kappa = 8*pi
E = function('E')(r,th)
P = function('P')(r)

s9 = Ee.subs(eps==0) == E
s10 = Pe.subs(eps==0) == P
sustituciones = (s1,s2,s3,s4,s5,s6,s7,s8,s9,s10)
Tdd.apply_map(lambda cmp: cmp.subs(sustituciones))
E_tt = solve(eins0[0,0].expr()==kappa*Tdd[0,0].expr(),E)[0].rhs()                                            #Eq 3.7

P_rr = solve(eins0[1,1].expr()==kappa*Tdd[1,1].expr(),P)[0].rhs()                                            #Eq 3.8
P_thth = solve(eins0[2,2].expr()==kappa*Tdd[2,2].expr(),P)[0].rhs()
P_phph = solve(eins0[3,3].expr()==kappa*Tdd[3,3].expr(),P)[0].rhs()
ddnu = solve(P_rr == P_thth, diff(nu,r,r))[0].rhs()                                                          #Eq 3.9

In [42]:
show(LatexExpr("E = "), E_tt)
show(LatexExpr("P_{rr} = "), P_rr)
show(LatexExpr("\\nu'' = "), ddnu)

E =  1/8*(r*diff(lamb(r), r) + e^lamb(r) - 1)*e^(-lamb(r))/(pi*r^2)

P_{rr} =  1/8*(r*diff(nu(r), r) - e^lamb(r) + 1)*e^(-lamb(r))/(pi*r^2)

\nu'' =  -1/2*(r^2*diff(nu(r), r)^2 - 2*r*diff(lamb(r), r) - (r^2*diff(lamb(r), r) + 2*r)*diff(nu(r), r) + 4*e^lamb(r) - 4)/r^2

In [43]:
dlamb = solve(E==E_tt,diff(lamb,r))[0].rhs()                                                                 #Eq 3.10
dnu = solve(P==P_rr,diff(nu,r))[0].rhs()                                                                     #Eq 3.11

In [44]:
show(LatexExpr("\\lambda' = "), dlamb)
show(LatexExpr("\\nu' = "), dnu)

\lambda' =  (8*pi*r^2*E(r, th)*e^lamb(r) - e^lamb(r) + 1)/r

\nu' =  (8*pi*r^2*P(r)*e^lamb(r) + e^lamb(r) - 1)/r

In [45]:
h0 = function('h0', latex_name="h_0")(r,th)
k0 = function('k0', latex_name="k_0")(r,th)
m0 = function('m0', latex_name="m_0")(r,th)

h2 = function('h2', latex_name="h_2")(r,th)
k2 = function('k2', latex_name="k_2")(r,th)
m2 = function('m2', latex_name="m_2")(r,th)

In [46]:
E1 = function('E1', latex_name='E^{(1)}')(r,th)
E2 = function('E2', latex_name='E^{(2)}')(r,th)
P1 = function('P1', latex_name='P^{(1)}')(r,th)
P2 = function('P2', latex_name='P^{(2)}')(r,th)

s11 = diff(Ee,eps).subs(eps==0) == E1
s12 = diff(Ee,eps,eps).subs(eps==0) == E2
s13 = diff(Pe,eps).subs(eps==0) == P1
s14 = diff(Pe,eps,eps).subs(eps==0) == P2

sustituciones = (s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14)

In [47]:
dTdd = M.tensor_field(0,2,'dTdd')
ddTdd = M.tensor_field(0,2,'ddTdd')
for i in range(0,n+1):
    for j in range(0,n+1):
        dTdd[i,j] = diff(Tedd[i,j].expr(),eps)
        ddTdd[i,j] = diff(Tedd[i,j].expr(),eps,eps)
dTdd.apply_map(lambda cmp: cmp.subs(sustituciones))                                              #d/de(T_\mu\nu)|e=0
ddTdd.apply_map(lambda cmp: cmp.subs(sustituciones))                                         #d^2/de^2(T_\mu\nu)|e=0

In [48]:
# Calculamos ahora la primera derivada de la ecuación

show(deins[1,1],LatexExpr(" = "), kappa*dTdd[1,1])
show(deins[2,2],LatexExpr(" = "), kappa*dTdd[2,2])
show(deins[3,3],LatexExpr(" = "), kappa*dTdd[3,3])

P1value = solve(deins[1,1].expr() == kappa*dTdd[1,1].expr(),P1)[0].rhs()

show(LatexExpr("\\text{De aquí sacamos }P^{1}(r)\\rightarrow\\text{$P^1(r)$ = }"), P1value)

0  =  8*pi*(2*eps^2*e^lamb(r)*m(r, th)*d(Pe)/deps + 4*eps*Pe(r, eps)*e^lamb(r)*m(r, th) + e^lamb(r)*d(Pe)/deps)

0  =  8*pi*(2*eps^2*r^2*k(r, th)*d(Pe)/deps + 4*eps*r^2*Pe(r, eps)*k(r, th) + r^2*d(Pe)/deps)

0  =  1/32*pi*((pi - 40*pi*LP2(th))^4*(r^4*Ne(r, eps)^2*Omegaevalue(eps)^2*d(Ee)/deps + 2*r^4*Ee(r, eps)*Ne(r, eps)*Omegaevalue(eps)^2*d(Ne)/deps + 2*r^4*Ne(r, eps)*Omegaevalue(eps)^2*Pe(r, eps)*d(Ne)/deps + 2*r^4*Ee(r, eps)*Ne(r, eps)^2*Omegaevalue(eps)*d(Omegaevalue)/deps + 2*r^4*Ne(r, eps)^2*Omegaevalue(eps)*Pe(r, eps)*d(Omegaevalue)/deps + r^4*Ne(r, eps)^2*Omegaevalue(eps)^2*d(Pe)/deps + 4*(eps^4*r^4*Ne(r, eps)^2*Omegaevalue(eps)^2*d(Ee)/deps + 2*eps^4*r^4*Ee(r, eps)*Ne(r, eps)*Omegaevalue(eps)^2*d(Ne)/deps + 2*eps^4*r^4*Ne(r, eps)*Omegaevalue(eps)^2*Pe(r, eps)*d(Ne)/deps + 2*eps^4*r^4*Ee(r, eps)*Ne(r, eps)^2*Omegaevalue(eps)*d(Omegaevalue)/deps + 2*eps^4*r^4*Ne(r, eps)^2*Omegaevalue(eps)*Pe(r, eps)*d(Omegaevalue)/deps + eps^4*r^4*Ne(r, eps)^2*Omegaevalue(eps)^2*d(Pe)/deps + 4*eps^3*r^4*Ee(r, eps)*Ne(r, eps)^2*Omegaevalue(eps)^2 + 4*eps^3*r^4*Ne(r, eps)^2*Omegaevalue(eps)^2*Pe(r, eps))*k(r, th)^2 + (eps^2*r^4*Ne(r, eps)^2*d(Ee)/deps + 2*eps^2*r^4*Ee(r, eps)*Ne(r, eps)*d(Ne)/deps + 2*eps^2*r^4*Ne(r, eps)*Pe(r, eps)*d(Ne)/deps + eps^2*r^4*Ne(r, eps)^2*d(Pe)/deps + 2*eps*r^4*Ee(r, eps)*Ne(r, eps)^2 + 2*eps*r^4*Ne(r, eps)^2*Pe(r, eps))*omega(r, th)^2 + 4*(eps^2*r^4*Ne(r, eps)^2*Omegaevalue(eps)^2*d(Ee)/deps + 2*eps^2*r^4*Ee(r, eps)*Ne(r, eps)*Omegaevalue(eps)^2*d(Ne)/deps + 2*eps^2*r^4*Ne(r, eps)*Omegaevalue(eps)^2*Pe(r, eps)*d(Ne)/deps + 2*eps^2*r^4*Ee(r, eps)*Ne(r, eps)^2*Omegaevalue(eps)*d(Omegaevalue)/deps + 2*eps^2*r^4*Ne(r, eps)^2*Omegaevalue(eps)*Pe(r, eps)*d(Omegaevalue)/deps + eps^2*r^4*Ne(r, eps)^2*Omegaevalue(eps)^2*d(Pe)/deps + 2*eps*r^4*Ee(r, eps)*Ne(r, eps)^2*Omegaevalue(eps)^2 + 2*eps*r^4*Ne(r, eps)^2*Omegaevalue(eps)^2*Pe(r, eps))*k(r, th) + 2*(eps*r^4*Ne(r, eps)^2*Omegaevalue(eps)*d(Ee)/deps + 2*eps*r^4*Ee(r, eps)*Ne(r, eps)*Omegaevalue(eps)*d(Ne)/deps + 2*eps*r^4*Ne(r, eps)*Omegaevalue(eps)*Pe(r, eps)*d(Ne)/deps + eps*r^4*Ee(r, eps)*Ne(r, eps)^2*d(Omegaevalue)/deps + eps*r^4*Ne(r, eps)^2*Pe(r, eps)*d(Omegaevalue)/deps + eps*r^4*Ne(r, eps)^2*Omegaevalue(eps)*d(Pe)/deps + r^4*Ee(r, eps)*Ne(r, eps)^2*Omegaevalue(eps) + r^4*Ne(r, eps)^2*Omegaevalue(eps)*Pe(r, eps) + 2*(eps^3*r^4*Ne(r, eps)^2*Omegaevalue(eps)*d(Ee)/deps + 2*eps^3*r^4*Ee(r, eps)*Ne(r, eps)*Omegaevalue(eps)*d(Ne)/deps + 2*eps^3*r^4*Ne(r, eps)*Omegaevalue(eps)*Pe(r, eps)*d(Ne)/deps + eps^3*r^4*Ee(r, eps)*Ne(r, eps)^2*d(Omegaevalue)/deps + eps^3*r^4*Ne(r, eps)^2*Pe(r, eps)*d(Omegaevalue)/deps + eps^3*r^4*Ne(r, eps)^2*Omegaevalue(eps)*d(Pe)/deps + 3*eps^2*r^4*Ee(r, eps)*Ne(r, eps)^2*Omegaevalue(eps) + 3*eps^2*r^4*Ne(r, eps)^2*Omegaevalue(eps)*Pe(r, eps))*k(r, th))*omega(r, th)) + 16*(pi - 40*pi*LP2(th))^2*(2*eps^2*r^2*k(r, th)*d(Pe)/deps + 4*eps*r^2*Pe(r, eps)*k(r, th) + r^2*d(Pe)/deps))

IndexError: list index out of range

In [ ]:
show(deins[0,0],LatexExpr(" = "), kappa*dTdd[0,0])

E1value = solve(deins[0,0].expr() == kappa*dTdd[0,0].expr(),E1)[0].rhs()
E1value = E1value.subs(sustituciones)
show(LatexExpr("\\text{De aquí sacamos }E^{(1)}(r)\\rightarrow\\text{$E^{(1)}(r)$ = }"), E1value)

In [ ]:
# DEMOSTRAR QUE ESTO ES LA ECUACIÓN 48 DE 2015
show(deins[0,3].expr() == kappa*dTdd[0,3].expr())                                                           #Eq 48

In [ ]:
# Para la segunda derivada tenemos

show(ddeins[0,0].expr(),LatexExpr(" = "), kappa*ddTdd[0,0].expr())

In [ ]:
show(ddeins[1,1],LatexExpr(" = "), kappa*ddTdd[1,1])

In [ ]:
show(ddeins[2,2],LatexExpr(" = "), kappa*ddTdd[2,2])

In [ ]:
show(ddeins[3,3],LatexExpr(" = "), kappa*ddTdd[3,3])

In [ ]:
show(ddeins[0,3],LatexExpr(" = "), kappa*ddTdd[0,3])

In [ ]:
show(ddeins[1,2],LatexExpr(" = "), kappa*ddTdd[1,2])

In [ ]:
# Aplicamos barotropic EoS
# Utilizando (6.11) de Reina con P1=0 y E1=0
E2*diff(P,r) - P2*diff(E,r) == 0

In [ ]:
prueba1 = M.tensor_field(0,2,'prueba1')
prueba2 = M.tensor_field(0,2,'prueba2')
for i in range(0,n+1):
    for j in range(0,n+1):
        prueba1[i,j] = ddeins[i,j]
        prueba2[i,j] = ddTdd[i,j]

In [ ]:
LP1 = function('LP1')(th)
LP2 = function('LP2')(th)
s0 = sin(th)^4 == (30*LP1^2-3)/35+1-2*LP1^2
s1 = sin(th)^2 == (2-2*LP2)/3
s2 = cos(th) == LP1
s3 = LP1^2 == (2*LP2+1)/3
s4 = sin(th)*cos(th) == 3*pi/16*LP1
s5 = sin(th) == pi/4-10*pi*LP2
su = (s0,s1,s2,s3,s4,s5)
prueba1.apply_map(lambda cmp: cmp.subs(su))
prueba2.apply_map(lambda cmp: cmp.subs(su))

In [ ]:
fm(r) = m0 + m2*LP2
fk(r) = k0 + k2*LP2
fh(r) = h0 + h2*LP2

prueba1.apply_map(lambda cmp: cmp.substitute_function(function('h'), fh))
prueba1.apply_map(lambda cmp: cmp.substitute_function(function('m'), fm))
prueba1.apply_map(lambda cmp: cmp.substitute_function(function('k'), fk))

prueba2.apply_map(lambda cmp: cmp.substitute_function(function('h'), fh))
prueba2.apply_map(lambda cmp: cmp.substitute_function(function('m'), fm))
prueba2.apply_map(lambda cmp: cmp.substitute_function(function('k'), fk))

In [ ]:
E02 = function('E02', latex_name='E_{0}^{(2)}')(r,th)
E12 = function('E12', latex_name='E_{1}^{(2)}')(r,th)
E22 = function('E22', latex_name='E_{2}^{(2)}')(r,th)

P02 = function('P02', latex_name='P_{0}^{(2)}')(r,th)
P12 = function('P12', latex_name='P_{1}^{(2)}')(r,th)
P22 = function('P22', latex_name='P_{2}^{(2)}')(r,th)

In [ ]:
fE2(r,th) = E02 + E12*LP1 + E22*LP2
fP2(r,th) = P02 + P12*LP1 + P22*LP2

prueba1.apply_map(lambda cmp: cmp.subs(LP1==0))
prueba1.apply_map(lambda cmp: cmp.subs(LP2==0))

prueba2.apply_map(lambda cmp: cmp.subs(LP1==0))
prueba2.apply_map(lambda cmp: cmp.subs(LP2==0))

prueba1.apply_map(lambda cmp: cmp.subs(E1==E1value))
prueba1.apply_map(lambda cmp: cmp.subs(P1==P1value))
prueba2.apply_map(lambda cmp: cmp.subs(E1==E1value))
prueba2.apply_map(lambda cmp: cmp.subs(P1==P1value))

prueba2.apply_map(lambda cmp: cmp.substitute_function(function('E2'), fE2))
prueba2.apply_map(lambda cmp: cmp.substitute_function(function('P2'), fP2))

prueba2.apply_map(lambda cmp: cmp.subs(u20==u20value))
prueba2.apply_map(lambda cmp: cmp.subs(u10==u10value))

In [ ]:
prueba2[0,0].expr()

In [38]:
prueba2[0,0].expr()

-1/8*pi^2*((2*r^2*E(r, th) + r^2*P(r))*omega(r, th)^2 + 2*(r^2*E(r, th)*Omega1(eps) + r^2*Omega1(eps)*P(r))*omega(r, th)) + 2*((r^2*Omega1(eps)^2 + 2*r^2*Omega1(eps)*omega(r, th) + r^2*omega(r, th)^2)*sin(th)^2 - 2*e^nu(r)*h(r, th))*E(r, th) + 2*((r^2*Omega1(eps)^2 + 2*r^2*Omega1(eps)*omega(r, th) + r^2*omega(r, th)^2)*sin(th)^2 - 2*e^nu(r)*h(r, th))*P(r) + E2(r, th)*e^nu(r) + 4*(2*E(r, th)*e^nu(r) + P(r)*e^nu(r))*h0(r, th)

In [39]:
prueba2[0,0].expr()

-1/8*pi^2*((2*r^2*E(r, th) + r^2*P(r))*omega(r, th)^2 + 2*(r^2*E(r, th)*Omega1(eps) + r^2*Omega1(eps)*P(r))*omega(r, th)) + 2*((r^2*Omega1(eps)^2 + 2*r^2*Omega1(eps)*omega(r, th) + r^2*omega(r, th)^2)*sin(th)^2 - 2*e^nu(r)*h(r, th))*E(r, th) + 2*((r^2*Omega1(eps)^2 + 2*r^2*Omega1(eps)*omega(r, th) + r^2*omega(r, th)^2)*sin(th)^2 - 2*e^nu(r)*h(r, th))*P(r) + E2(r, th)*e^nu(r) + 4*(2*E(r, th)*e^nu(r) + P(r)*e^nu(r))*h0(r, th)

In [40]:
E1value = solve(prueba1[0,0].expr() == kappa*prueba2[0,0].expr(),E1)[0].rhs()

IndexError: list index out of range

In [ ]:
E02 = solve(prueba1[0,0].expr() == kappa*prueba2[0,0].expr(),E02);E02

In [ ]:
prueba1[0,0]

In [ ]:
legendre_P(2,cos(th))

In [ ]:
eq1 = LP2==legendre_P(2,cos(th))
solve(eq1,cos(th)^2)[0].rhs()

In [ ]:
eq1 = LP2==legendre_P(2,cos(th))
solve(eq1,sin(th))[0].rhs()

In [ ]:
fm(r) = m0 + m2*(3*cos(th)^2-1)/2
fk(r) = k0 + k2*(3*cos(th)^2-1)/2
fh(r) = h0 + h2*(3*cos(th)^2-1)/2

ddeins.apply_map(lambda cmp: cmp.substitute_function(function('h'), fh))
ddeins.apply_map(lambda cmp: cmp.substitute_function(function('m'), fm))
ddeins.apply_map(lambda cmp: cmp.substitute_function(function('k'), fk))